<a href="https://colab.research.google.com/github/KAIET98/TFM_AGROTECH/blob/TBA_GOOGLE_FUNC/REPLICA_BASEDATOS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Objetivo: 

El objetivo de este script es construir un script general que sirva para generalizar la subida al nuevo servidor d ela base de datos. 

## Librerias

In [7]:
!pip install pymysql


In [8]:

import pandas as pd
import numpy as np
import pymysql
#import cryptography
import pandas as pd
import matplotlib.pyplot as plt
import pymysql

In [9]:
#el nuevo


host ='34.79.75.171'
#host = '35.241.159.127'
user = 'admin'
password = '12345678'
database = 'GET_DATABASE'


connection = pymysql.connect(host=host,
                             user=user,
                             password=password,
                             db=database)

cursor = connection.cursor()
#cursor=db.cursor()

cursor.execute("SHOW TABLES")


#Sacamos una primera lista para guardar los nombres de las tablas que tenemos en la base de datos de pago

tablas = []

for table_name in cursor:
  # print(table_name[0])

   tablas.append(table_name[0])
   
#finlamente seleccionamos solamente las validas. y las guardamos en otra
#lista independiente

tablas_nombre = []

for tabla in tablas[1:5]: 
  tablas_nombre.append(tabla)

#y finalmente la ukltima tabla valida

tablas_nombre.append(tablas[-1])

print('Estas son las tablas que tenemos: ')

tablas_nombre

Estas son las tablas que tenemos: 


['DATOS_METEREOLOGIA_HISTORICO_TBA',
 'HUMEDAD',
 'HUMEDAD_prueba',
 'METEREOLOGIA',
 'TBA_AGROSENSORS_HISTORIC_TRAINING']

## Cargamos las funciones que nos permitiran replicar lo que tenemos hasta ahora

In [10]:
def run_query(q):
    with pymysql.connect(host=host,
                             user=user,
                             password=password,
                             db=database) as conn:
        return pd.read_sql(q, conn)


def insert_into(data, arrival_df):

    #establecemos la conexión a la base de datos
    connection = pymysql.connect(host=host,
                             user=user,
                             password=password,
                             db=database)

    cursor = connection.cursor()


    #especificamos cual es el nombre de la tabla destino

    nombre_tabla = arrival_df

    
    # creating column list for insertion
    cols = "`,`".join([str(i) for i in data.columns.tolist()])
    cols = cols.strip()
    # Insert DataFrame recrds one by one.
    for i,row in data.iterrows():
        sql = "INSERT INTO `{}` (`".format(nombre_tabla) +cols + "`) VALUES (" + "%s,"*(len(row)-1) + "%s)"

            
        cursor.execute(sql, tuple(row))

            # the connection is not autocommitted by default, so we must commit to save our changes
        connection.commit()


def checkTableExists(conn, tablename):
    dbcur = conn.cursor()
    dbcur.execute("""
        SELECT COUNT(*)
        FROM information_schema.tables
        WHERE table_name = '{0}'
        """.format(tablename.replace('\'', '\'\'')))
    if dbcur.fetchone()[0] == 1:
        dbcur.close()
        return True

    dbcur.close()
    return False

In [11]:
import logging
import requests

logger = logging.getLogger()
logger.setLevel(logging.INFO)

#host ='34.79.75.171'  #este ees el host viejo
host = '35.241.159.127' #este el el host nuevo
user = 'admin'
password = '12345678'
database = 'GET_DATABASE'

try:
    conn = pymysql.connect(host=host, user=user, passwd=password, db=database, connect_timeout=5)
except pymysql.MySQLError as e:
    logger.error("ERROR: Unexpected error: Could not connect to MySQL instance.")
    logger.error(e)
    #sys.exit()

logger.info("SUCCESS: Connection to RDS MySQL instance succeeded")


print(tablas_nombre)

for tabla in tablas_nombre: 

  print(tabla)

  
  #host ='34.79.75.171'  #este ees el host viejo
  host = '35.241.159.127' #este el el host nuevo
  user = 'admin'
  password = '12345678'
  database = 'GET_DATABASE'


  try:
    conn = pymysql.connect(host=host, user=user, passwd=password, db=database, connect_timeout=5)
  except pymysql.MySQLError as e:
    logger.error("ERROR: Unexpected error: Could not connect to MySQL instance.")
    logger.error(e)
      #sys.exit()

  logger.info("SUCCESS: Connection to RDS MySQL instance succeeded")

    

  if checkTableExists(conn, tabla) != True:


    #Primero lanzamos la consulta de select all a la base d edatos vieja
    #sobre esa tabla

    
    host ='34.79.75.171' #este ees el host viejo
    #host = '35.241.159.127' #este el el host nuevo
    user = 'admin'
    password = '12345678'
    database = 'GET_DATABASE'
    
    
    logger = logging.getLogger()
    logger.setLevel(logging.INFO)

    try:
        conn = pymysql.connect(host=host, user=user, passwd=password, db=database, connect_timeout=5)
    except pymysql.MySQLError as e:
        logger.error("ERROR: Unexpected error: Could not connect to MySQL instance.")
        logger.error(e)
        #sys.exit()

    logger.info("SUCCESS: Connection to RDS MySQL instance succeeded")

    q = 'select * from {};'.format(tabla)

    tabla_df = run_query(q)


    #nos conectamos a la nueva base de datos

    print('\n', 'Creamos la tabla: {}'.format(tabla))


    # Dependiendo de una tabla u otra tendremos que crear un esquema para albergar
    #los datos

    #host ='34.79.75.171' #este ees el host viejo
    host = '35.241.159.127' #este el el host nuevo
    user = 'admin'
    password = '12345678'
    database = 'GET_DATABASE'
    
    
    logger = logging.getLogger()
    logger.setLevel(logging.INFO)

    try:
        conn = pymysql.connect(host=host, user=user, passwd=password, db=database, connect_timeout=5)
    except pymysql.MySQLError as e:
        logger.error("ERROR: Unexpected error: Could not connect to MySQL instance.")
        logger.error(e)
        #sys.exit()

    logger.info("SUCCESS: Connection to RDS MySQL instance succeeded")

    #1. Humedad_prueba

    if tabla == 'HUMEDAD_prueba':
      
      q = '''
        CREATE TABLE HUMEDAD_prueba(
        id_humedad  INT  PRIMARY KEY AUTO_INCREMENT, 
        Humedad_relativa int, 
        DIA DATE NOT NULL, 
        HORA TIME
        );
        '''

      print(run_query(q))

      print('Tabla creada, insertamos datos', '\n')

      print('... insertando datos')

      insert_into(tabla, tabla_df)

      print('\n', 'tabla {}'.format(tabla), 'ha sido insertada...', ' siguiente tabla: ', '\n')



    #2. Humedad


    if tabla == 'HUMEDAD':
      
      q = '''
        CREATE TABLE HUMEDAD(
        id_humedad  INT  PRIMARY KEY AUTO_INCREMENT, 
        Humedad_relativa int, 
        DIA DATE NOT NULL, 
        HORA TIME
        );
        '''

      print(run_query(q))

      print('Tabla creada, insertamos datos', '\n')

      print('... insertando datos')

      insert_into(tabla, tabla_df)

      print('\n', 'tabla {}'.format(tabla), 'ha sido insertada...', ' siguiente tabla: ', '\n')



    #2. Historico de metereologia de la estacion de albacete

    if tabla == 'METEREOLOGIA':

      q = '''CREATE TABLE METEREOLOGIA(

      fecha   DATE NOT NULL, 
      Estacion VARCHAR(255) NOT NULL, 
      Provincia VARCHAR(255), 
      Temperatura_media INT,
      Precipitacion_l_m3 INT, 
          Temperatura_minima float, 
          Temperatura_maxima float,
          Direccion_viento VARCHAR(255), 
          Velocidad_media float, 
          Presion_maxima float,
            Presion_minima float, 
            Prom_temperatura_media_prov float, 
            Grados_debajo_siete float,
            Presion_externa float
            
            );'''


      print(run_query(q))

      print('Tabla creada, insertamos datos', '\n')

      print('... insertando datos')

      insert_into(tabla, tabla_df)

      print('\n', 'tabla {}'.format(tabla), 'ha sido insertada...', ' siguiente tabla: ', '\n')

    #3. HISTORICO, VALENCIA COORDENADAS

    if tabla == 'DATOS_METEREOLOGIA_HISTORICO_TBA':

      q = '''CREATE TABLE {}(

      fecha   DATE NOT NULL, 
      Estacion VARCHAR(255) NOT NULL, 
      Provincia VARCHAR(255), 
      Temperatura_media INT,
      Precipitacion_l_m3 INT, 
          Temperatura_minima float, 
          Temperatura_maxima float,
          Direccion_viento VARCHAR(255), 
          Velocidad_media float, 
          Presion_maxima float,
            Presion_minima float, 
            Prom_temperatura_media_prov float, 
            Grados_debajo_siete float,
            Presion_externa float
            
            );'''.format(tabla)


      print(run_query(q))

      print('Tabla creada, insertamos datos', '\n')

      print('... insertando datos')

      insert_into(tabla, tabla_df)

      print('\n', 'tabla {}'.format(tabla), 'ha sido insertada...', ' siguiente tabla: ', '\n')

    
    #Finalmente con los datos de la sensorica
    
    if tabla == 'TBA_AGROSENSORS_HISTORIC_TRAINING':

      q = """CREATE TABLE {}(

      id_nodo VARCHAR(255), 
      HUMEDAD_1 integer, 
      HUMEDAD_2 integer, 
      HUMEDAD_3 integer, 
      HUMEDAD_4 integer, 
      HUMEDAD_5 integer, 
      HUMEDAD_6 integer, 
      COLUMNA_7 integer, 
      TEMPERATURA FLOAT, 
      HUMEDAD_RELATIVA FLOAT, 
      VELOCIDAD_VIENTO FLOAT, 
      LLUVIA FLOAT, 
      DIRECCION_VIENTO INTEGER,
      HORA TIME,
      FECHA TIMESTAMP(6)

      )

      """.format(tabla)

      print(run_query(q))

      print('Tabla creada, insertamos datos', '\n')

      print('... insertando datos')

      insert_into(tabla, tabla_df)

      print('\n', 'tabla {}'.format(tabla), 'ha sido insertada...', ' siguiente tabla: ', '\n')

    

    



      

    #print()


    '''
    host ='34.79.75.171'
    #host = '35.241.159.127'
    user = 'admin'
    password = '12345678'
    database = 'GET_DATABASE'
    '''

     

  else:

    q = 'select * from {};'.format(tabla)

    tabla_df = run_query(q)


    if tabla_df.shape[0] != 0:

      print('La tabla ya ha sido relleanda con anterioridad :)')

    else:




      #Primero lanzamos la consulta de select all a la base d edatos vieja
      #sobre esa tabla

      
      host ='34.79.75.171' #este ees el host viejo
      #host = '35.241.159.127' #este el el host nuevo
      user = 'admin'
      password = '12345678'
      database = 'GET_DATABASE'
      
      
      logger = logging.getLogger()
      logger.setLevel(logging.INFO)

      try:
          conn = pymysql.connect(host=host, user=user, passwd=password, db=database, connect_timeout=5)
      except pymysql.MySQLError as e:
          logger.error("ERROR: Unexpected error: Could not connect to MySQL instance.")
          logger.error(e)
          #sys.exit()

      logger.info("SUCCESS: Connection to RDS MySQL instance succeeded")

      q = 'select * from {};'.format(tabla)

      tabla_df = run_query(q)


      #nos conectamos a la nueva base de datos

      #print('\n', 'Creamos la tabla: {}'.format(tabla))


      # Dependiendo de una tabla u otra tendremos que crear un esquema para albergar
      #los datos

      #host ='34.79.75.171' #este ees el host viejo
      host = '35.241.159.127' #este el el host nuevo
      user = 'admin'
      password = '12345678'
      database = 'GET_DATABASE'
      
      
      logger = logging.getLogger()
      logger.setLevel(logging.INFO)

      try:
          conn = pymysql.connect(host=host, user=user, passwd=password, db=database, connect_timeout=5)
      except pymysql.MySQLError as e:
          logger.error("ERROR: Unexpected error: Could not connect to MySQL instance.")
          logger.error(e)
          #sys.exit()

      logger.info("SUCCESS: Connection to RDS MySQL instance succeeded")
      
      print('Tabla creada, insertamos datos', '\n')

      print('... insertando datos')


      #para la tablas de la humedad y humedad prueba, los datos de SQL nos son 
      #devueltos en un formato no muy amigable para hacer el reformateo por lo 
      #que hacemos un pequeño cambio extrayendo los digitos d ela hora, aunque ste paso deba ser algo 
      #obvio; no lo es.

      if tabla == "HUMEDAD":

        #primero converitmos la columna en string para poder manipularlo

        tabla_df.HORA = tabla_df.HORA.map(lambda x: str(x))


        #luego pasamos un pattern para poder sacar los digitos de la hora
        tabla_df['HORA'] = tabla_df.HORA.str.extract('(\d+\:\d+\:\d+)')


        print(tabla_df.head())


      if tabla == "HUMEDAD_prueba":

        #primero converitmos la columna en string para poder manipularlo

        tabla_df.HORA = tabla_df.HORA.map(lambda x: str(x))


        #luego pasamos un pattern para poder sacar los digitos de la hora
        tabla_df['HORA'] = tabla_df.HORA.str.extract('(\d+\:\d+\:\d+)')


        print(tabla_df.head())


      if tabla == 'TBA_AGROSENSORS_HISTORIC_TRAINING': 

        #primero converitmos la columna en string para poder manipularlo

        tabla_df.HORA = tabla_df.HORA.map(lambda x: str(x))


        #luego pasamos un pattern para poder sacar los digitos de la hora
        tabla_df['HORA'] = tabla_df.HORA.str.extract('(\d+\:\d+\:\d+)')


        print(tabla_df.head())


      insert_into(tabla_df, tabla)

      print('\n', 'tabla {}'.format(tabla), 'ha sido insertada...', ' siguiente tabla: ', '\n')
    


  

['DATOS_METEREOLOGIA_HISTORICO_TBA', 'HUMEDAD', 'HUMEDAD_prueba', 'METEREOLOGIA', 'TBA_AGROSENSORS_HISTORIC_TRAINING']
DATOS_METEREOLOGIA_HISTORICO_TBA
La tabla ya ha sido relleanda con anterioridad :)
HUMEDAD
La tabla ya ha sido relleanda con anterioridad :)
HUMEDAD_prueba
La tabla ya ha sido relleanda con anterioridad :)
METEREOLOGIA
La tabla ya ha sido relleanda con anterioridad :)
TBA_AGROSENSORS_HISTORIC_TRAINING
Tabla creada, insertamos datos 

... insertando datos
  id_nodo  HUMEDAD_1  HUMEDAD_2  HUMEDAD_3  HUMEDAD_4  HUMEDAD_5  HUMEDAD_6  \
0       1          0          0          0          0          0          0   
1       1          0          0          0          0          0          0   
2       1          0          0          0          0          0          0   
3       1          0          0          0          0          0          0   
4       1          0          0          0          0          0          0   

   COLUMNA_7  TEMPERATURA  HUMEDAD_RELATIVA  VELO